# This notebook demonstrates the workflow of creating a cloud-free Landsat-9 imagery composite from Microsoft Planetary Computer for the Island of Hawaii

The workflow includes the following steps:
1. Find all the Landsat-9 scenes from summer of 2022 that covers Hawaii
2. Create a RasterCollection from all the scenes
3. Use a custom function to remove cloud pixels from all the Landsat images
4. Use the median function to find the best pixel from all overlapping images

#### Estimated running time: 10 minutes

### Import relevant modules and create a RasterCollection of Landsat-9 images from planetary computer archive 

In [ ]:
import arcpy
from arcpy import AIO

In [ ]:
# Create an AIO object from the cloud storage connection file for data access
a = AIO(r'C:\AMPC_Resources\ACS_Files\esrims_pc_landsat-c2-l2.acs')

In [ ]:
# Define the query to search for images
query = {
    "collections": ["landsat-c2-l2"], # Landsat collection 2 level 2 product (https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2)
    "bbox": [-156.1271927, 18.8710955, -154.7755921, 20.2990569], # Define bounding box over Hawaii
    "query": {"platform": {"in": ["landsat-9"]}}, # Define query to only return Landsat-9 images
    "datetime": "2022-01-01/2022-12-31", # Define time range
    "limit": 1000, # Define max number of results to be returned
          }

**Certain files may be corrupted in the MPC collection so some datasets may be skipped**

In [ ]:
# Create a RasterCollection object that contains the search results
rc = arcpy.ia.RasterCollection.fromSTACAPI(stac_api="https://planetarycomputer.microsoft.com/api/stac/v1",
                                  query=query,
                                  attribute_dict={
                                      "Name":"id",
                                      "Cloud Cover":"eo:cloud_cover",
                                      "StdTime":"datetime",
                                      "Platform":"platform",
                                      "Spatial Reference":"proj:epsg",
                                      "Extent": "bbox",
                                    })

In [ ]:
# Visualize the RasterCollection
rc

### Define a function to remove cloud pixels and compute cloud-free composite

In [ ]:
# Define function to remove cloud pixels based on QA band
def remove_cloud(item):
    raster = item['Raster']
    qa_band = raster.getRasterBands(['QA'])
    
    cloud_mask = arcpy.ia.TransposeBits(qa_band, [0,1,2,3,4], [0,1,2,3,4], 0, None)
    value_mask = ~cloud_mask
    
    cloud_free_raster = arcpy.ia.Clip(raster, aoi = value_mask)
    
    return {'raster': cloud_free_raster, "Name": item["Name"], "AcquisitionDate": item["StdTime"]}

In [ ]:
# Apply the above function to remove cloud pixels from each image in the RasterCollection
rc_cloud_free = rc.map(remove_cloud)

In [ ]:
# Apply the Median function to find the most representative pixels value from overlapping images
cloud_free_composite_median = rc_cloud_free.median(ignore_nodata = True, extent_type = 'UnionOf')

In [ ]:
# Display the cloud free image composite
cloud_free_composite_median